# OpenAI Agents SDK with Custom MCP Server for CSV Querying

## Overview

This notebook demonstrates how to use OpenAI's Agents SDK with a custom Model Context Protocol (MCP) server to query data from a CSV file.

### What is the OpenAI Responses API?

The **Responses API** is OpenAI's newest and most advanced API (released March 2025) that combines the strengths of:
- **Chat Completions API** - For conversational interactions
- **Assistants API** - For stateful, multi-turn conversations with tools

Key features:
- Stateful conversations with automatic context management
- Built-in tool support (file search, web search, code interpreter)
- Native MCP (Model Context Protocol) server integration
- Support for remote and local MCP servers

### OpenAI Agents SDK

The **Agents SDK** is a Python framework built on top of the Responses API that provides:
- Easy agent creation and orchestration
- Simple integration with MCP servers via stdio transport
- Automatic tool discovery and execution
- Multi-agent workflows

### MCP (Model Context Protocol)

MCP is an open standard (introduced by Anthropic, adopted by OpenAI in March 2025) that standardizes how AI models interact with external tools and data sources.

**Transport Types:**
- **stdio** - Local subprocess communication (what we'll use here)
- **HTTP/SSE** - Remote server communication (for deployed services)

### What We'll Build

In this demo, we'll:
1. Create a custom MCP server that provides tools to query CSV data
2. Connect to it using stdio transport (local subprocess)
3. Use OpenAI's Agents SDK to interact with the data naturally through conversation

## Setup and Installation

First, let's install the required dependencies:

```bash
pip install openai-agents mcp pandas
```

Make sure you have your OpenAI API key set:

```bash
export OPENAI_API_KEY='your-api-key-here'
```

In [1]:
import asyncio
import os
from agents import Agent, Runner
from agents.mcp import MCPServerStdio
import pandas as pd

# Verify API key is set
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️  Warning: OPENAI_API_KEY not set!")
    print("Please set it with: export OPENAI_API_KEY='your-key'")
else:
    print("✅ OpenAI API key is set")

✅ OpenAI API key is set


## Our Sample Data

Let's first look at the CSV data we'll be querying:

In [2]:
# Load and display the sample CSV data
df = pd.read_csv('sample_data.csv')
print(f"Total products: {len(df)}\n")
df.head(10)

Total products: 15



,product_id,product_name,category,price,stock,rating
0,1,Laptop Pro 15,Electronics,1299.99,45,4.5
1,2,Wireless Mouse,Electronics,29.99,150,4.2
2,3,Office Chair Deluxe,Furniture,349.99,30,4.7
3,4,Standing Desk,Furniture,599.99,20,4.6
4,5,USB-C Hub,Electronics,49.99,200,4.3
5,6,Mechanical Keyboard,Electronics,149.99,75,4.8
6,7,Monitor 27inch,Electronics,399.99,60,4.4
7,8,Desk Lamp LED,Furniture,79.99,100,4.1
8,9,Ergonomic Mouse Pad,Electronics,24.99,250,4.0
9,10,Webcam HD,Electronics,89.99,80,4.5


## Understanding the MCP Server

Our custom MCP server (`csv_query_mcp_server.py`) provides several tools:

1. **get_all_products()** - Retrieve all products
2. **search_products_by_category(category)** - Filter by category
3. **search_products_by_price_range(min_price, max_price)** - Filter by price
4. **get_product_by_name(product_name)** - Search by name
5. **get_top_rated_products(limit)** - Get highest-rated items
6. **get_products_in_stock(min_stock)** - Check inventory
7. **get_category_statistics()** - Get aggregated stats

The server is built using **FastMCP**, which provides a simple decorator-based API for creating MCP tools.

## Creating the Agent with MCP Server Integration

Now let's create an OpenAI agent that connects to our MCP server via stdio transport.

### How stdio Transport Works:

1. **MCPServerStdio** launches our Python script as a subprocess
2. Communication happens through stdin/stdout pipes
3. The SDK handles all the MCP protocol details automatically
4. Tools are discovered via the `tools/list` MCP endpoint
5. Tool calls are executed through `tools/call` MCP endpoint

In [3]:
async def create_agent_with_mcp():
    """
    Create an agent connected to our CSV query MCP server.
    """
    # Connect to the MCP server using stdio transport
    async with MCPServerStdio(
        name="CSV Query Server",
        params={
            "command": "python",
            "args": ["csv_query_mcp_server.py"]
        },
    ) as server:
        # Create an agent with access to the MCP server's tools
        agent = Agent(
            name="Product Assistant",
            model="gpt-5-mini",  # or "gpt-4o", "gpt-5", etc.
            instructions="""
            You are a helpful product assistant with access to a product catalog.
            Use the available tools to answer questions about products, including:
            - Searching by category, price, or name
            - Finding top-rated products
            - Checking inventory levels
            - Providing category statistics
            
            Always provide clear, helpful responses based on the data.
            """,
            mcp_servers=[server],  # Connect the MCP server to the agent
        )
        
        return agent, server

# Note: We'll use this function in the examples below

## Example 1: Simple Product Search

Let's ask the agent to find products in a specific category.

In [4]:
from tabnanny import verbose


async def run_agentic_search(query):
    async with MCPServerStdio(
        name="CSV Query Server",
        params={
            "command": "python",
            "args": ["csv_query_mcp_server.py"]
        },
    ) as server:
        agent = Agent(
            name="Product Assistant",
            model="gpt-5-mini",
            instructions="You are a helpful product assistant. Use the tools to answer questions.",
            mcp_servers=[server],
        )
        
        # Query the agent
        print(f"Query: {query}\n")
        
        result = await Runner.run(starting_agent=agent, input=query)
        print(f"Response:\n{result.final_output}")

# Run the example
query = "What electronics products do we have?"
await run_agentic_search(query)

Query: What electronics products do we have?

Response:
We have 9 electronics products:

- Laptop Pro 15 — $1,299.99 — stock: 45 — rating: 4.5  
- Wireless Mouse — $29.99 — stock: 150 — rating: 4.2  
- USB-C Hub — $49.99 — stock: 200 — rating: 4.3  
- Mechanical Keyboard — $149.99 — stock: 75 — rating: 4.8  
- Monitor 27inch — $399.99 — stock: 60 — rating: 4.4  
- Ergonomic Mouse Pad — $24.99 — stock: 250 — rating: 4.0  
- Webcam HD — $89.99 — stock: 80 — rating: 4.5  
- Cable Organizer — $15.99 — stock: 300 — rating: 4.2  
- Portable SSD 1TB — $129.99 — stock: 90 — rating: 4.7

Total units in stock (electronics): 1,250.  
Would you like details on any specific product or to filter by price/stock/rating?


In [5]:
df

,product_id,product_name,category,price,stock,rating
0,1,Laptop Pro 15,Electronics,1299.99,45,4.5
1,2,Wireless Mouse,Electronics,29.99,150,4.2
2,3,Office Chair Deluxe,Furniture,349.99,30,4.7
3,4,Standing Desk,Furniture,599.99,20,4.6
4,5,USB-C Hub,Electronics,49.99,200,4.3
5,6,Mechanical Keyboard,Electronics,149.99,75,4.8
6,7,Monitor 27inch,Electronics,399.99,60,4.4
7,8,Desk Lamp LED,Furniture,79.99,100,4.1
8,9,Ergonomic Mouse Pad,Electronics,24.99,250,4.0
9,10,Webcam HD,Electronics,89.99,80,4.5


## Example 2: Price Range Query

Let's search for products within a specific price range.

In [6]:
# Run the example
query = "Show me products that cost between $50 and $150"
await run_agentic_search(query)

Query: Show me products that cost between $50 and $150

Response:
Here are the products priced between $50 and $150:

- Mechanical Keyboard — Electronics — $149.99 — Stock: 75 — Rating: 4.8  
- Portable SSD 1TB — Electronics — $129.99 — Stock: 90 — Rating: 4.7  
- Webcam HD — Electronics — $89.99 — Stock: 80 — Rating: 4.5  
- Desk Lamp LED — Furniture — $79.99 — Stock: 100 — Rating: 4.1

Would you like details for any specific product (specs, images), or to filter by category or rating?


In [7]:
df[(df['price'] >= 50) & (df['price'] <= 150)]

,product_id,product_name,category,price,stock,rating
5,6,Mechanical Keyboard,Electronics,149.99,75,4.8
7,8,Desk Lamp LED,Furniture,79.99,100,4.1
9,10,Webcam HD,Electronics,89.99,80,4.5
13,14,Portable SSD 1TB,Electronics,129.99,90,4.7


## Example 3: Top Rated Products

Let's find the highest-rated products.

In [8]:
query = "What are the top 3 highest-rated products?"
await run_agentic_search(query)

Query: What are the top 3 highest-rated products?

Response:
Here are the top 3 highest-rated products:

1. Mechanical Keyboard (product_id 6) — Category: Electronics — Price: $149.99 — Stock: 75 — Rating: 4.8  
2. Office Chair Deluxe (product_id 3) — Category: Furniture — Price: $349.99 — Stock: 30 — Rating: 4.7  
3. Portable SSD 1TB (product_id 14) — Category: Electronics — Price: $129.99 — Stock: 90 — Rating: 4.7

(Note: two products share the second-highest rating of 4.7.)


In [9]:
df.sort_values(by='rating', ascending=False).head(3)

,product_id,product_name,category,price,stock,rating
5,6,Mechanical Keyboard,Electronics,149.99,75,4.8
2,3,Office Chair Deluxe,Furniture,349.99,30,4.7
13,14,Portable SSD 1TB,Electronics,129.99,90,4.7


## Example 4: Category Statistics

Let's get aggregated statistics by category.

In [10]:
query = "Give me a summary of our product categories with average prices and ratings"
await run_agentic_search(query)

Query: Give me a summary of our product categories with average prices and ratings

Response:
Here’s a summary by category:

- Electronics — 9 products — average price $243.43 — average rating 4.40 out of 5 — total stock 1,250  
- Furniture — 6 products — average price $215.82 — average rating 4.45 out of 5 — total stock 490

Would you like this exported (CSV/JSON) or a breakdown of products within any category?


In [11]:
df.groupby("category").agg(
    avg_price=("price", "mean"),
    avg_rating=("rating", "mean"),
    count=("product_id", "count")
).reset_index()

,category,avg_price,avg_rating,count
0,Electronics,243.434444,4.40,9
1,Furniture,215.823333,4.45,6


## Example 5: Complex Multi-Step Query

The agent can use multiple tools in sequence to answer complex questions.

In [14]:
async def example_complex_query():
    async with MCPServerStdio(
        name="CSV Query Server",
        params={
            "command": "python",
            "args": ["csv_query_mcp_server.py"]
        },
    ) as server:
        agent = Agent(
            name="Product Assistant",
            model="gpt-4o-mini",
            instructions="You are a helpful product assistant. Use the tools to answer questions.",
            mcp_servers=[server],
        )
        
        query = """
        I need to buy some office equipment. Can you help me find:
        1. A good keyboard (check ratings)
        2. Any furniture items under $200
        3. Tell me if they're in stock
        """
        print(f"Query: {query}\n")
        
        result = await Runner.run(starting_agent=agent, input=query)
        print(f"Response:\n{result.final_output}")

# Run the example
await example_complex_query()

Query: 
        I need to buy some office equipment. Can you help me find:
        1. A good keyboard (check ratings)
        2. Any furniture items under $200
        3. Tell me if they're in stock
        

Response:
Here are the results for your search for office equipment:

### 1. Keyboard
- **Mechanical Keyboard**
  - **Category**: Electronics
  - **Price**: $149.99
  - **Stock**: 75
  - **Rating**: 4.8/5

### 2. Furniture Items Under $200
- **Desk Lamp LED**
  - **Price**: $79.99
  - **Stock**: 100
  - **Rating**: 4.1/5

- **Laptop Stand**
  - **Price**: $39.99
  - **Stock**: 120
  - **Rating**: 4.6/5

- **Desk Mat Large**
  - **Price**: $34.99
  - **Stock**: 180
  - **Rating**: 4.4/5

### Summary
- **Keyboard**: In stock and highly rated.
- **Furniture Items**: All items listed are under $200 and are in stock. 

Let me know if you need further assistance!


## How It Works: Behind the Scenes

### MCP Protocol Flow

When you run the agent:

1. **Server Initialization**
   - `MCPServerStdio` launches `csv_query_mcp_server.py` as a subprocess
   - Communication pipes (stdin/stdout) are established

2. **Tool Discovery**
   - The SDK calls the MCP `tools/list` endpoint
   - The server returns all available tools with their schemas
   - This information is cached in an `mcp_list_tools` context item

3. **Agent Reasoning**
   - User query is sent to the OpenAI model (via Responses API)
   - Model sees the available tools in its context
   - Model decides which tool(s) to call based on the query

4. **Tool Execution**
   - SDK sends tool call request to MCP server via `tools/call`
   - Server executes the tool (e.g., queries the CSV)
   - Results are returned through the pipe

5. **Response Generation**
   - Tool results are added to the conversation context
   - Model generates a natural language response
   - Final output is returned to the user

### Responses API vs. Agents SDK

**Responses API (Direct):**
```python
# Direct API call (for remote MCP servers)
response = client.responses.create(
    model="gpt-4o",
    tools=[{
        "type": "mcp",
        "server_url": "https://your-server.com/mcp",
        "server_label": "my-server"
    }],
    input="Your query"
)
```

**Agents SDK (Higher-level):**
```python
# Agents SDK (supports stdio and remote)
async with MCPServerStdio(...) as server:
    agent = Agent(
        model="gpt-4o",
        mcp_servers=[server]
    )
    result = await Runner.run(agent, "Your query")
```

The Agents SDK provides:
- Easier local development with stdio
- Automatic subprocess management
- Multi-agent orchestration
- Simplified async handling

## Key Takeaways

### Benefits of MCP

1. **Standardization** - One protocol for all tool integrations
2. **Simplicity** - Decorator-based tool creation with FastMCP
3. **Flexibility** - Works locally (stdio) or remotely (HTTP/SSE)
4. **Composability** - Combine multiple MCP servers in one agent

### When to Use stdio vs. HTTP

**stdio (Local)**:
- Development and testing
- Single-user applications
- No network required
- Process isolation

**HTTP/SSE (Remote)**:
- Production deployments
- Multi-user services
- Centralized tool management
- Cross-network access

### Best Practices

1. **Tool Design**
   - Keep tools focused and single-purpose
   - Provide clear descriptions for the model
   - Return structured, parseable data

2. **Error Handling**
   - Return meaningful error messages
   - Handle edge cases gracefully
   - Validate inputs

3. **Performance**
   - Use tool filtering when possible
   - Cache expensive operations
   - Optimize data queries

4. **Security**
   - Validate and sanitize inputs
   - Limit file system access
   - Use appropriate authentication for remote servers

## Interactive Demo

Try your own queries! This cell creates an interactive chat loop.

In [13]:
async def interactive_demo():
    """
    Interactive demo - ask your own questions!
    Type 'exit' or 'quit' to stop.
    """
    async with MCPServerStdio(
        name="CSV Query Server",
        params={
            "command": "python",
            "args": ["csv_query_mcp_server.py"]
        },
    ) as server:
        agent = Agent(
            name="Product Assistant",
            model="gpt-4o-mini",
            instructions="""
            You are a helpful product assistant with access to a product catalog.
            Use the available tools to answer questions about products.
            Always provide clear, helpful responses based on the data.
            """,
            mcp_servers=[server],
        )
        
        print("Interactive Product Assistant")
        print("Type 'exit' or 'quit' to stop\n")
        print("Example queries:")
        print("- What's the cheapest laptop?")
        print("- Show me all furniture items")
        print("- Which products have the best ratings?\n")
        
        while True:
            try:
                # In Jupyter, we'll use input()
                query = input("Your question: ")
                
                if query.lower() in ['exit', 'quit', '']:
                    print("Goodbye!")
                    break
                
                result = await Runner.run(starting_agent=agent, input=query)
                print(f"\nAssistant: {result.final_output}\n")
                
            except KeyboardInterrupt:
                print("\nGoodbye!")
                break
            except Exception as e:
                print(f"Error: {e}\n")

# Uncomment to run the interactive demo
# await interactive_demo()

## Next Steps

Now that you understand the basics, you can:

1. **Extend the MCP Server**
   - Add more tools (insert, update, delete)
   - Support multiple CSV files
   - Add data validation and error handling

2. **Deploy as HTTP Server**
   - Convert to SSE or Streamable HTTP
   - Add authentication
   - Use with Responses API directly

3. **Build Multi-Agent Systems**
   - Combine multiple MCP servers
   - Create specialized agents for different tasks
   - Implement agent handoffs

4. **Add Other Tools**
   - Combine with web_search_preview
   - Add code_interpreter for analysis
   - Integrate with other MCP servers

### Resources

- [OpenAI Agents SDK Docs](https://openai.github.io/openai-agents-python/)
- [MCP Specification](https://spec.modelcontextprotocol.io/)
- [OpenAI Cookbook - MCP Examples](https://cookbook.openai.com/examples/mcp/mcp_tool_guide)
- [FastMCP Documentation](https://github.com/modelcontextprotocol/python-sdk)